# Carga de Datos

In [ ]:
import pandas as pd
import pandas as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
kof_produ = pd.read_csv('DF_3(sin_outliers).csv', encoding='latin-1',index_col=1)

<ipython-input-33-4755dbdd3937>:1: DtypeWarning: Columns (182) have mixed types. Specify dtype option on import or set low_memory=False.
  kof_produ = pd.read_csv('DF_3(sin_outliers).csv', encoding='latin-1',index_col=1)


In [ ]:
kofi_produ = pd.read_csv('DF_3(sin_outliers).csv', encoding='latin-1')

<ipython-input-34-6ee7553a16c8>:1: DtypeWarning: Columns (182) have mixed types. Specify dtype option on import or set low_memory=False.
  kofi_produ = pd.read_csv('DF_3(sin_outliers).csv', encoding='latin-1')


In [ ]:
kof_produc = pd.read_csv('df_productos.csv', encoding='latin-1',index_col=0)

In [ ]:
kof_produc.shape

(16288, 35)

In [ ]:
kof_produ.isna().sum()

In [ ]:
kofi_produ.drop(['Otros', 'Unnamed: 0'	], axis= 1, inplace=True)

In [ ]:
kofi_produ.head()

In [ ]:
prod_kofi= kofi_produ.iloc[:, 0:165]

In [ ]:
prod_kofi

In [ ]:
prod_kof= kof_produ.iloc[:, 1:166]

In [ ]:
prod_kof.drop('Otros', axis= 1, inplace=True)

In [ ]:
prod_kof

### Estandarizado con MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
estandarizado_kof = scaler.fit_transform(prod_kof)

###### Clase para Centroides y Plots

In [ ]:
def centroide(num_cluster, datos, clusters):
    ind = clusters == num_cluster
    return(pd.DataFrame(datos[ind].mean()).T)

def recodificar(col, nuevo_codigo):
    col_cod = pd.Series(col, copy=True)
    for llave, valor in nuevo_codigo.items():
        col_cod.replace(llave, valor, inplace=True)
    return col_cod

def bar_plot(centros, labels, scale = False,cluster = None, var = None):
    fig, ax = plt.subplots(1,1, figsize = (15,8), dpi = 200)
    
    centros = np.copy(centros)
    
    if scale:
        for col in range(centros.shape[1]):
            centros[:,col] /= max(centros[:,col])
    
    colores = color_palette()
    minimo = floor(centros.min()) if floor(centros.min()) < 0 else 0
    def inside_plot(valores, labels, titulo):
        plt.barh(range(len(valores)), valores, 1/1.5, color = colores)
        plt.xlim(minimo, ceil(centros.max()))
        plt.title(titulo)
    if var is not None:
        centros = np.array([n[[x in var for x in labels]] for n in centros])
        colores = [colores[x % len(colores)] for x, i in enumerate(labels) if i in var]
        labels = labels[[x in var for x in labels]]
    if cluster is None:
        for i in range(centros.shape[0]):
            plt.subplot(1, centros.shape[0], i + 1)
            inside_plot(centros[i].tolist(), labels, ('Cluster ' + str(i)))
            plt.yticks(range(len(labels)), labels) if i == 0 else plt.yticks([]) 
    else:
        pos = 1
        for i in cluster:
            plt.subplot(1, len(cluster), pos)
            inside_plot(centros[i].tolist(), labels, ('Cluster ' + str(i)))
            plt.yticks(range(len(labels)), labels) if pos == 1 else plt.yticks([]) 
            pos += 1
            
def bar_plot_detail(centros,columns_names = [], columns_to_plot = [],figsize = (10,7),dpi = 150):
  fig, ax = plt.subplots(1,1, figsize = (15,8), dpi = 200)
  numClusters = centros.shape[0]
  labels = ["Cluster "+ str(i) for i in range(numClusters)]
  centros = pd.DataFrame(centros,columns=columns_names,index= labels)
  
  plots = len(columns_to_plot) if len(columns_to_plot) != 0 else len(columns_names)
  rows, cols = ceil(plots/2),2
  
  plt.figure(1, figsize = figsize,dpi = dpi)
  plt.subplots_adjust(hspace=1,wspace = 0.5)
  columns = columns_names
  if len(columns_to_plot) > 0: 
    if type(columns_to_plot[0]) is str:
      columns = columns_to_plot
    else:
      columns = [columns_names[i] for i in columns_to_plot]
  var = 0
  for numRow in range(rows):
    for numCol in range(cols):
      if var < plots:
        ax = plt.subplot2grid((rows, cols), (numRow, numCol), colspan=1, rowspan=1)
        sb.barplot(y = labels, x=columns[var] ,data=centros ,ax=ax)
        var += 1    
              
def radar_plot(centros, labels):
    
    fig, ax = plt.subplots(1,1, figsize = (15,8), dpi = 200)
    centros = np.array([((n - min(n)) / (max(n) - min(n)) * 100) if 
                        max(n) != min(n) else (n/n * 50) for n in centros.T])
    angulos = [n / float(len(labels)) * 2 * pi for n in range(len(labels))]
    angulos += angulos[:1]
    ax = plt.subplot(111, polar = True)
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    
    plt.xticks(angulos[:-1], labels)
    ax.set_rlabel_position(0)
    plt.yticks([10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
           ["10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%", "100%"], 
           color = "grey", size = 8)
    plt.ylim(-10, 100)
    for i in range(centros.shape[1]):
        valores = centros[:, i].tolist()
        valores += valores[:1]
        ax.plot(angulos, valores, linewidth = 1, linestyle = 'solid', 
                label = 'Cluster ' + str(i))
        ax.fill(angulos, valores, alpha = 0.3)
    plt.legend(loc='upper right', bbox_to_anchor = (0.1, 0.1))

#### Distancias

In [ ]:
from scipy.spatial.distance import pdist

In [ ]:
distancias_prod = pdist(estandarizado_kof, metric="euclidean")
distancias_prod

array([0.24900962, 0.19656659, 0.17612989, ..., 0.08181874, 0.20586039,
       0.1864551 ])

##### Clase para Gráfico de Barras

In [ ]:
def bar_plot(centros, labels, scale = False,cluster = None, var = None):
    from math import ceil, floor
    from seaborn import color_palette
    
    centros = np.copy(centros)
    
    if scale:
        for col in range(centros.shape[1]):
            centros[:,col] /= max(centros[:,col])
    
    colores = color_palette()
    minimo = floor(centros.min()) if floor(centros.min()) < 0 else 0
    def inside_plot(valores, labels, titulo):
        plt.barh(range(len(valores)), valores, 1/1.5, color = colores)
        plt.xlim(minimo, ceil(centros.max()))
        plt.title(titulo)
    if var is not None:
        centros = np.array([n[[x in var for x in labels]] for n in centros])
        colores = [colores[x % len(colores)] for x, i in enumerate(labels) if i in var]
        labels = labels[[x in var for x in labels]]
    if cluster is None:
        for i in range(centros.shape[0]):
            plt.subplot(1, centros.shape[0], i + 1)
            inside_plot(centros[i].tolist(), labels, ('Cluster ' + str(i)))
            plt.yticks(range(len(labels)), labels) if i == 0 else plt.yticks([]) 
    else:
        pos = 1
        for i in cluster:
            plt.subplot(1, len(cluster), pos)
            inside_plot(centros[i].tolist(), labels, ('Cluster ' + str(i)))
            plt.yticks(range(len(labels)), labels) if pos == 1 else plt.yticks([]) 
            pos += 1

##### Centroide

In [ ]:
def centroide(num_cluster, datos, clusters):
    ind = clusters == num_cluster
    return(pd.DataFrame(estandarizado_kof[ind].mean()).T)

## Cluster

#### Índice

In [ ]:
kof_indice= pd.read_csv('df_productos.csv', encoding='latin-1')

In [ ]:
kof_indice= kof_indice['KUNNR']

#### Grupos (Clústers)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, ward, single, complete,average,linkage, fcluster

Con 10 K

In [ ]:
grupos_10 = fcluster(linkage(estandarizado_kof, method = 'ward', metric='euclidean'), 10, criterion = 'maxclust')
grupos_10 = grupos_10-1 
grupos_10

array([3, 9, 9, ..., 9, 9, 9], dtype=int32)

### Etiquetado

####Emparejamiento de KUNNR con clúster

In [ ]:
etiquetado_grupos_10 = dict(zip(kof_indice, grupos_10))

# Imprimir el etiquetado de los grupos
print(etiquetado_grupos_10)

{1300005503: 3, 1300006528: 9, 1300006533: 9, 1300201177: 9, 1300223225: 9, 1300224271: 9, 1300229608: 9, 1300234221: 9, 1300234272: 9, 1300250715: 9, 1300253955: 9, 1300253956: 9, 1320003678: 9, 1320004269: 9, 1320005852: 9, 1320007848: 9, 1320012945: 9, 1320026478: 9, 1320034295: 9, 1320034335: 9, 1320036121: 9, 1300005498: 9, 1300006531: 9, 1300015800: 9, 1300016606: 5, 1300203315: 9, 1300229607: 3, 1320003799: 9, 1320005817: 9, 1320007356: 9, 1320012814: 4, 1320012886: 9, 1320027099: 9, 1320037865: 9, 1320038373: 9, 1300006526: 9, 1300224601: 9, 1300229016: 9, 1320005023: 9, 1320014260: 9, 1320037111: 9, 1300230761: 9, 1300234322: 9, 1320005260: 3, 1320025188: 3, 1320030107: 9, 1300007690: 9, 1300009009: 3, 1300016714: 9, 1300228524: 9, 1300230789: 9, 1300253756: 9, 1320000336: 9, 1320006531: 9, 1320013506: 9, 1320017616: 9, 1320038304: 9, 1320038991: 9, 1300007688: 5, 1300007689: 9, 1300007691: 9, 1300234220: 9, 1320004294: 3, 1320005836: 9, 1320006557: 9, 1320014051: 9, 132001584

###### Encontrar todos los clientes que pertenecen a un cluster específico

In [ ]:
grupo_deseado = 9

# Buscar los nombres de usuario del grupo deseado
usuarios_grupo_9 = [nombre_usuario for nombre_usuario, grupo in etiquetado_grupos_10.items() if grupo == grupo_deseado]

# Imprimir los nombres de usuario del grupo 3
print(usuarios_grupo_9)

[1300006528, 1300006533, 1300201177, 1300223225, 1300224271, 1300229608, 1300234221, 1300234272, 1300250715, 1300253955, 1300253956, 1320003678, 1320004269, 1320005852, 1320007848, 1320012945, 1320026478, 1320034295, 1320034335, 1320036121, 1300005498, 1300006531, 1300015800, 1300203315, 1320003799, 1320005817, 1320007356, 1320012886, 1320027099, 1320037865, 1320038373, 1300006526, 1300224601, 1300229016, 1320005023, 1320014260, 1320037111, 1300230761, 1300234322, 1320030107, 1300007690, 1300016714, 1300228524, 1300230789, 1300253756, 1320000336, 1320006531, 1320013506, 1320017616, 1320038304, 1320038991, 1300007689, 1300007691, 1300234220, 1320005836, 1320006557, 1320014051, 1320016624, 1320019185, 1320026244, 1320033490, 1320034810, 1300007696, 1300202686, 1320000109, 1320000116, 1320012756, 1320013108, 1320013146, 1320013548, 1320016546, 1320021049, 1320021623, 1320022251, 1320027551, 1320036599, 1320037498, 1320038855, 1320038981, 1300006102, 1300006103, 1300058187, 1300058188, 130

###### Contador de individuos por clúster

In [ ]:
from collections import Counter

# Contar la cantidad de usuarios por grupo
conteo_usuarios_por_grupo_10 = Counter(etiquetado_grupos_10.values())

# Imprimir el conteo de usuarios por grupo
for grupo, cantidad in conteo_usuarios_por_grupo_10.items():
    print(f"Grupo {grupo}: {cantidad} usuarios")

Grupo 3: 878 usuarios
Grupo 9: 13987 usuarios
Grupo 5: 561 usuarios
Grupo 4: 260 usuarios
Grupo 6: 369 usuarios
Grupo 8: 178 usuarios
Grupo 7: 33 usuarios
Grupo 1: 6 usuarios
Grupo 2: 10 usuarios
Grupo 0: 3 usuarios


# Creación de nuevo DF con Cluster 3

In [ ]:
import pandas as pd

# Crear una lista de tuplas (clave, valor) para los elementos que tienen puntuación 8
elementos = [(k, v) for k, v in etiquetado_grupos_10.items() if v == 1]

# Crear un nuevo DataFrame a partir de la lista de elementos
df_k_1 = pd.DataFrame(elementos, columns=['KUNNR', 'Cluster'])

# Imprimir el nuevo DataFrame
print(df_k_1)

        KUNNR  Cluster
0  1300052380        1
1  1300010672        1
2  1300018550        1
3  1300010668        1
4  1300056237        1
5  1320004873        1


In [ ]:
import pandas as pd

# Crear una lista de tuplas (clave, valor) para los elementos que tienen puntuación 8
elementos_9 = [(k, v) for k, v in etiquetado_grupos_10.items() if v == 9]

# Crear un nuevo DataFrame a partir de la lista de elementos
df_k_9 = pd.DataFrame(elementos_9, columns=['KUNNR', 'Cluster'])

# Imprimir el nuevo DataFrame
df_k_9

,KUNNR,Cluster
0,1300006528,9
1,1300006533,9
2,1300201177,9
3,1300223225,9
4,1300224271,9
...,...,...
13982,1400021830,9
13983,1400021843,9
13984,1400059468,9
13985,1400056310,9


In [ ]:
df_original = prod_kof


df_nueva = df_k_1['KUNNR']


df_unido = df_original.merge(df_nueva, on='KUNNR', how='inner')


df_unido

Union de bases con el cluster 9 (Más grande)

In [ ]:
df_original = prod_kof


df_nueva_9 = df_k_9['KUNNR']


df_cluster9 = df_original.merge(df_nueva_9, on='KUNNR', how='inner')


df_cluster9

# Reconfiguración de la tabla

In [ ]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 26.9 MB/s eta 0:00:00


In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import implicit

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


### Train y Test

In [ ]:
from sklearn.model_selection import train_test_split

# División aleatoria de los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(df_unido, test_size=0.2, random_state=42)

### Transformación de la tabla

In [ ]:
df_cluster9

In [ ]:
kun_pro_9 = pd.DataFrame(df_cluster9)

In [ ]:
kun_pro_9.set_index('KUNNR', inplace=True)

In [ ]:
# Transformar la tabla
producto_kun_9 = kun_pro_9.stack().reset_index()  # Convertir las columnas de productos en filas
producto_kun_9.columns = ['KUNNR', 'Producto', 'Consumo']  # Renombrar las columnas

producto_kun_9

In [ ]:
producto_kun_9['Consumo'] = producto_kun_9['Consumo'].replace(0, np.nan)

# Eliminar las filas que contengan valores nulos
producto_kun_9 = producto_kun_9.dropna()

# Restablecer los índices del DataFrame
producto_kun_9 = producto_kun_9.reset_index(drop=True)

# Imprimir el DataFrame resultante
producto_kun_9

,KUNNR,Producto,Consumo
0,1300006528,ADES_946 ml_TETRAPACK_NR,0.9996
1,1300006528,AGUA CIEL_1.2 Litros_PET_NR,5.0721
2,1300006528,AGUA CIEL_338.1 Onz / 10 LITROS_PET_NR,1.7611
3,1300006528,AGUA CIEL_50.7 Onz / 1.5 LITROS_PET_NR,6.3402
4,1300006528,AMEYAL_67.6 Onz / 2 LITROS_PET_NR,1.4089
...,...,...,...
510532,1400056310,SPRITE_20.3 Onz / 600 ML=CC_PET_NR,13.3140
510533,1400082675,CC_12 Onz / 355/354 ML=CC_ALUMINIO_NR,326.3763
510534,1400082675,PREDATOR_16 Onz / 473 ML=CC_ALUMINIO_NR,23.6661
510535,1400082675,SIDRAL_12 Onz / 355/354 ML=CC_ALUMINIO_NR,26.5006


In [ ]:
productos_unicos_9 = producto_kun_9['Producto'].unique()

# Asignar un índice único a cada producto
indices_productos_9 = pd.factorize(productos_unicos_9)[0]

# Crear un diccionario de mapeo entre nombres de productos y sus índices
mapeo_productos_9 = dict(zip(productos_unicos_9, indices_productos_9))

# Agregar una columna 'ID_Producto' al DataFrame original con los índices únicos de los productos
producto_kun_9['ID_Producto'] = producto_kun_9['Producto'].map(mapeo_productos_9)

# Imprimir el DataFrame resultante
producto_kun_9

### Analysis

In [ ]:
print(f'Number of unique customers: {producto_kun_9.KUNNR.nunique()}')
print(f'Number of unique items: {producto_kun_9.Producto.nunique()}')
print(f'Number of unique items: {producto_kun_9.ID_Producto.nunique()}')
print(f'Average purchase quantity per interaction: {int(producto_kun_9.Consumo.mean())}')
print(f'Minimum purchase quantity per interaction: {producto_kun_9.Consumo.min()}')
print(f'Maximum purchase quantity per interaction: {producto_kun_9.Consumo.max()}')

Number of unique customers: 13973
Number of unique items: 162
Number of unique items: 162
Average purchase quantity per interaction: 65
Minimum purchase quantity per interaction: 0.044
Maximum purchase quantity per interaction: 307172.2155


In [ ]:
unique_customers = producto_kun_9.KUNNR.unique()
customer_ids = dict(zip(unique_customers, np.arange(unique_customers.shape[0], dtype=np.int32)))

unique_items = producto_kun_9.ID_Producto.unique()
item_ids = dict(zip(unique_items, np.arange(unique_items.shape[0], dtype=np.int32)))

producto_kun_9['customer_id'] = producto_kun_9.KUNNR.apply(lambda i: customer_ids[i])
producto_kun_9['item_id'] = producto_kun_9.ID_Producto.apply(lambda i: item_ids[i])

sparse_item_customer = sparse.csr_matrix((producto_kun_9['Consumo'].astype(float), (producto_kun_9['item_id'], producto_kun_9['customer_id'])))
sparse_customer_item = sparse.csr_matrix((producto_kun_9['Consumo'].astype(float), (producto_kun_9['customer_id'], producto_kun_9['item_id'])))

In [ ]:
producto_kun_9.drop('ID_Producto', axis= 1, inplace= True)

In [ ]:
producto_kun_9

,KUNNR,Producto,Consumo,customer_id,item_id
0,1300006528,ADES_946 ml_TETRAPACK_NR,0.9996,0,0
1,1300006528,AGUA CIEL_1.2 Litros_PET_NR,5.0721,0,1
2,1300006528,AGUA CIEL_338.1 Onz / 10 LITROS_PET_NR,1.7611,0,2
3,1300006528,AGUA CIEL_50.7 Onz / 1.5 LITROS_PET_NR,6.3402,0,3
4,1300006528,AMEYAL_67.6 Onz / 2 LITROS_PET_NR,1.4089,0,4
...,...,...,...,...,...
510532,1400056310,SPRITE_20.3 Onz / 600 ML=CC_PET_NR,13.3140,13971,32
510533,1400082675,CC_12 Onz / 355/354 ML=CC_ALUMINIO_NR,326.3763,13972,8
510534,1400082675,PREDATOR_16 Onz / 473 ML=CC_ALUMINIO_NR,23.6661,13972,29
510535,1400082675,SIDRAL_12 Onz / 355/354 ML=CC_ALUMINIO_NR,26.5006,13972,119


In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=10, regularization=0.1, iterations=500)

alpha = 30
data = (sparse_item_customer * alpha).astype('double')

model.fit(data)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
producto_kun_9.loc[producto_kun_9['item_id'] == 75].head()

# Creación de Dataframe con Productos Estrella por KUNNR

### SetUp de la base general

In [ ]:
prod_f = pd.DataFrame(prod_kofi)

In [ ]:
prod_f

In [ ]:
prod_f.set_index('KUNNR', inplace=True)

In [ ]:
# Transformar la tabla
producto_star = prod_f.stack().reset_index()  # Convertir las columnas de productos en filas
producto_star.columns = ['KUNNR', 'Producto', 'Consumo']  # Renombrar las columnas

producto_star

In [ ]:
productos_unicos_star = producto_star['Producto'].unique()

# Asignar un índice único a cada producto
indices_productos_star = pd.factorize(productos_unicos_star)[0]

# Crear un diccionario de mapeo entre nombres de productos y sus índices
mapeo_productos_star = dict(zip(productos_unicos_star, indices_productos_star))

# Agregar una columna 'ID_Producto' al DataFrame original con los índices únicos de los productos
producto_star['ID_Producto'] = producto_star['Producto'].map(mapeo_productos_star)

In [ ]:
producto_star

In [ ]:
unique_customers_s = producto_star.KUNNR.unique()
customer_ids_s = dict(zip(unique_customers_s, np.arange(unique_customers_s.shape[0], dtype=np.int32)))

unique_items_s = producto_star.ID_Producto.unique()
item_ids_s = dict(zip(unique_items_s, np.arange(unique_items_s.shape[0], dtype=np.int32)))

producto_star['customer_id'] = producto_star.KUNNR.apply(lambda i: customer_ids_s[i])
producto_star['item_id'] = producto_star.ID_Producto.apply(lambda i: item_ids_s[i])

sparse_item_customer_s = sparse.csr_matrix((producto_star['Consumo'].astype(float), (producto_star['item_id'], producto_star['customer_id'])))
sparse_customer_item_s = sparse.csr_matrix((producto_star['Consumo'].astype(float), (producto_star['customer_id'], producto_star['item_id'])))

In [ ]:
producto_star.drop('ID_Producto', axis= 1, inplace= True)

In [ ]:
producto_star

,KUNNR,Producto,Consumo,customer_id,item_id
0,1300005503,ADES_6.8 Onz / 200 ML=CC_TETRAPACK_NR,0.1409,0,0
1,1300005503,ADES_946 ml_TETRAPACK_NR,0.4998,0,1
2,1300005503,AGUA CIEL_1.2 Litros_PET_NR,2.5361,0,2
3,1300005503,AGUA CIEL_12 Onz / 355/354 ML=CC_PET_NR,0.0000,0,3
4,1300005503,AGUA CIEL_169.1 Onz / 5 LITROS_PET_NR,3.5223,0,4
...,...,...,...,...,...
2670735,1420021839,TOPOCHI_50.7 Onz / 1.5 LITROS_PET_NR,0.0000,16284,159
2670736,1420021839,VALLE PULP_13.5 Onz / 400 ML=CC_PET_NR,0.0000,16284,160
2670737,1420021839,VITAMIN WA_16.9 Onz / 500 ML=CC_PET_NR,0.0000,16284,161
2670738,1420021839,YOLI_20.3 Onz / 600 ML=CC_PET_NR,0.0000,16284,162


### Set up del star product

In [ ]:
max_consumo_idx = producto_star.groupby('KUNNR')['Consumo'].idxmax()

# Seleccionar las filas correspondientes al máximo consumo por cliente
df_prod_star = producto_star.loc[max_consumo_idx]

# Imprimir el nuevo dataframe
df_prod_star

In [ ]:
df_prod_star.reset_index(inplace=True)

In [ ]:
df_prod_star

In [ ]:
df_prod_star.drop('index', axis= 1, inplace= True)

In [ ]:
print(f'Number of unique customers: {df_prod_star.KUNNR.nunique()}')
print(f'Number of unique items: {df_prod_star.Producto.nunique()}')
print(f'Minimum purchase quantity per interaction: {df_prod_star.Consumo.min()}')
print(f'Maximum purchase quantity per interaction: {df_prod_star.Consumo.max()}')

# Prueba de accuracy

In [ ]:
def calcular_precision_top_n(recomendaciones, interacciones_reales, n):
    total_precision = 0
    total_usuarios = 0

    for usuario, items_relevantes in interacciones_reales.items():
        recomendaciones_usuario = recomendaciones.get(usuario, [])
        recomendaciones_usuario_top_n = recomendaciones_usuario[:n]

        relevantes_en_top_n = len(set(recomendaciones_usuario_top_n) & set(items_relevantes))
        precision = relevantes_en_top_n / n

        total_precision += precision
        total_usuarios += 1

    precision_media = total_precision / total_usuarios

    return precision_media

# Prueba de modelo conectado

In [ ]:
df_prod_star

In [ ]:
import numpy as np

# Base de datos de productos más consumidos por cliente
productos_consumidos = df_prod_star  # Aquí debes cargar tu base de datos

In [ ]:
productos_consumidos['KUNNR'] = productos_consumidos['KUNNR'].astype(int)

In [ ]:
productos_consumidos.dtypes

In [ ]:
# Función para generar recomendaciones basadas en un producto
def generar_recomendaciones(item_id, n_similar):
    customer_vecs = model.item_factors
    item_vecs = model.user_factors
    item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

    scores = item_vecs.dot(item_vecs[item_id]) / item_norms
    top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
    similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

    for item in similar:
        idx, score = item
        recomendaciones = producto_kun_9.Producto.loc[producto_kun_9.item_id == idx].iloc[0]
        print(recomendaciones)
        print(f"Puntuación: {score}")
        print()

# Función para buscar el producto más consumido por un cliente específico
def buscar_producto_mas_consumido(cliente):
    producto_mas_consumido = None
    consumo_maximo = 0

    for producto in df_prod_star:
        if producto['KUNNR'] == int(cliente):  # Convertir cliente a cadena para la comparación
            if producto['Consumo'] > consumo_maximo:
                consumo_maximo = producto['Consumo']
                producto_mas_consumido = producto['item_id']

    if producto_mas_consumido is not None:
        generar_recomendaciones(producto_mas_consumido, 5)
    else:
        print("No se encontró el cliente en la base de datos de productos consumidos.")

In [ ]:
df_prod_star

# Prueba alternativa

In [ ]:
# Función para generar recomendaciones basadas en un producto
def generar_recomendaciones(item_id, n_similar):
    customer_vecs = model.item_factors
    item_vecs = model.user_factors
    item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

    scores = item_vecs.dot(item_vecs[item_id]) / item_norms
    top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
    similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

    for item in similar:
        idx, score = item
        recomendaciones = producto_kun_9.Producto.loc[producto_kun_9.item_id == idx].iloc[0]
        print(recomendaciones)
        print(f"Puntuación: {score}")
        print()

# Función para buscar el producto más consumido por un cliente específico
def buscar_producto_mas_consumido(cliente):
    df_cliente = df_prod_star[df_prod_star['KUNNR'] == int(cliente)]
    if not df_cliente.empty:
        producto_mas_consumido = df_cliente.loc[df_cliente['Consumo'].idxmax(), 'item_id']
        generar_recomendaciones(producto_mas_consumido, 10)
    else:
        print("No se encontró el cliente en la base de datos de productos consumidos.")

In [ ]:
buscar_producto_mas_consumido(1300223225)

In [ ]:
generar_recomendaciones(75, 10)

In [ ]:
recomendaciones= buscar_producto_mas_consumido(1300223225)

CC_59.2 Onz / 1.75 LITROS_PET_NR
Puntuación: 1.0000001192092896

FANTA_67.6 Onz / 2 LITROS_PET_NR
Puntuación: 0.9999984502792358

SIDRAL_101.4 Onz / 3 LITROS_PET_NR
Puntuación: 0.9999929666519165

DV FRUT_67.6 Onz / 2 LITROS_PET_NR
Puntuación: 0.9999891519546509

SPRITE_101.4 Onz / 3 LITROS_PET_NR
Puntuación: 0.999973475933075

FRESCA_101.4 Onz / 3 LITROS_PET_NR
Puntuación: 0.9999703168869019

SIDRAL_67.6 Onz / 2 LITROS_PET_NR
Puntuación: 0.9999445676803589

FRESCA_67.6 Onz / 2 LITROS_PET_NR
Puntuación: 0.9999302625656128

FANTA_101.4 Onz / 3 LITROS_PET_NR
Puntuación: 0.9998927712440491

CC_45.6 Onz / 1.35 LITROS_PET_NR
Puntuación: 0.9998771548271179



In [ ]:
!pip install nbconvert

In [ ]:
!jupyter nbconvert --to html Evidenica3.ipynb

[NbConvertApp] Converting notebook Evidenica3.ipynb to html


# Planteamiento de la solución

In [ ]:
import pandas as pd

In [ ]:
DF_3=pd.read_csv("DF_3_PRUEBA.csv")

<ipython-input-86-5841e5f53023>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  DF_3=pd.read_csv("DF_3_PRUEBA.csv")


In [ ]:
DF_3.dtypes

Unnamed: 0     float64
KUNNR           object
KUNNR2         float64
BZIRK           object
ZZCRMA_LONG    float64
                ...   
VPH_CVJ        float64
VPH_SINRTV     float64
VPH_SINLTC     float64
VPH_SINCINT    float64
VPH_SINTIC     float64
Length: 498, dtype: object

In [ ]:
Av = pd.read_csv("actividades.csv")

In [ ]:
Av.dtypes

Unnamed: 0          int64
KUNNR               int64
Centros             int64
Comercio mayoreo    int64
Comercio menor      int64
Escuela             int64
Fabricas            int64
Farmacias           int64
Restaurantes        int64
dtype: object

In [ ]:
Av['KUNNR'] = Av['KUNNR'].astype(str)

In [ ]:
Av.head()

,Unnamed: 0,KUNNR,Centros,Comercio mayoreo,Comercio menor,Escuela,Fabricas,Farmacias,Restaurantes
0,118,1300005503,0,0,0,0,0,0,0
1,203,1300006528,0,0,0,0,0,0,0
2,204,1300006533,0,0,0,0,0,0,0
3,2217,1300201177,0,0,0,0,0,0,0
4,2433,1300223225,0,0,0,0,0,0,0


In [ ]:
Av.shape

(16288, 9)

In [ ]:
DF_3['KUNNR'] = DF_3['KUNNR'].apply(lambda x: str(x).split('.')[0])

In [ ]:
DF_3 = DF_3.fillna(0)

In [ ]:
DF_3 = DF_3.drop(DF_3.index[-2:])

In [ ]:
# Definir la función obtener_datos_por_kunnr
def obtener_datos_por_kunnr(df, KUNNR):
    datos = {}
    filtro = df['KUNNR'] == KUNNR
    
    # Verificar si hay alguna fila que cumpla la condición
    if filtro.any():
        fila = df[filtro].iloc[0]

        datos['Urbano'] = fila['Urbano']
        datos['CITY1'] = fila['CITY1']
        datos['total'] = fila['total']
        datos['GM_2020'] = fila['GM_2020']
        datos['ZCAL_RM2'] = fila['ZCAL_RM2']
    else:
        print("No se encontraron datos para el valor de KUNNR proporcionado.")
    
    return datos

kunnr = input("Ingrese el valor de KUNNR: ")

datos_kunnr = obtener_datos_por_kunnr(DF_3, kunnr)

print(datos_kunnr)

Ingrese el valor de KUNNR: 1300201177
{'Urbano': 'Rural', 'CITY1': 'TOLUCA', 'total': 2120.8546, 'GM_2020': 'Bajo', 'ZCAL_RM2': 'Abts., Misc. y Minis'}


In [ ]:
def obtener_datos_x_kunnr(df, KUNNR):
    datos = {}
    fila = df[df['KUNNR'] == KUNNR].iloc[0]
    
    datos['Centros'] = fila['Centros']
    datos['Comercio mayoreo'] = fila['Comercio mayoreo']
    datos['Comercio menor'] = fila['Comercio menor']
    datos['Escuela'] = fila['Escuela']
    datos['Fabricas'] = fila['Fabricas']
    datos['Farmacias'] = fila['Farmacias']
    datos['Restaurantes'] = fila['Restaurantes']
    
    return datos

# Solicitar el valor de KUNNR al usuario
kunnr = input("Ingrese el valor de KUNNR: ")

# Llamar a la función obtener_datos_por_kunnr con los valores proporcionados
datos_kunnr = obtener_datos_x_kunnr(Av, kunnr)

print(datos_kunnr)

Ingrese el valor de KUNNR: 1420021839
{'Centros': 2, 'Comercio mayoreo': 1, 'Comercio menor': 25, 'Escuela': 0, 'Fabricas': 3, 'Farmacias': 0, 'Restaurantes': 10}


In [ ]:
# Definir la función obtener_datos_por_kunnr
def obtener_datos_por_kunnr(df, KUNNR):
    datos = {}
    filtro = df['KUNNR'] == KUNNR
    
    # Verificar si hay alguna fila que cumpla la condición
    if filtro.any():
        fila = df[filtro].iloc[0]
        # Obtener los datos específicos de la fila seleccionada
        datos['Urbano'] = fila['Urbano']
        datos['CITY1'] = fila['CITY1']
        datos['total'] = fila['total']
        datos['GM_2020'] = fila['GM_2020']
        datos['ZCAL_RM2'] = fila['ZCAL_RM2']
    else:
        print("No se encontraron datos para el valor de KUNNR proporcionado.")
    
    return datos

# Solicitar al usuario el valor de KUNNR
kunnr = input("Ingrese el valor de KUNNR: ")

# Llamar a la función obtener_datos_por_kunnr con los valores proporcionados
datos_kunnr = obtener_datos_por_kunnr(DF_3, kunnr)

print(datos_kunnr)
recomendaciones_resultado = buscar_producto_mas_consumido(kunnr)  # Llamar a la función recomendaciones
print(recomendaciones_resultado)

Ingrese el valor de KUNNR: 1300005503
{'Urbano': 'Rural', 'CITY1': 'TOLUCA', 'total': 3256.5825, 'GM_2020': 'Bajo', 'ZCAL_RM2': 'Otros HM'}
SIDRAL_84.5 Onz / 2.5 LITROS_REF PET_RETORNABLE
Puntuación: 1.0000001192092896

FANTA_84.5 Onz / 2.5 LITROS_REF PET_RETORNABLE
Puntuación: 0.9999964833259583

FRESCA_84.5 Onz / 2.5 LITROS_REF PET_RETORNABLE
Puntuación: 0.9999878406524658

DLAWARE_2 LITROS_PET_NR
Puntuación: 0.9999579191207886

FRESCA-S/A_67.6 Onz / 2 LITROS_PET_NR
Puntuación: 0.9999452829360962

CC_7.9 Onz / 235ml=CC_VIDRIO_RETORNABLE
Puntuación: 0.9998875856399536

MULTPRD_84.5 Onz / 2.5 LITROS_REF PET_RETORNABLE
Puntuación: 0.9998665452003479

DV FRUT_84.5 Onz / 2.5 LITROS_REF PET_RETORNABLE
Puntuación: 0.9998583793640137

CC_76.1 Onz / 2.25 LITROS_PET_NR
Puntuación: 0.9998430013656616

CC_45.6 Onz / 1.35 LITROS_PET_NR
Puntuación: 0.9997970461845398

None


In [ ]:
import matplotlib.pyplot as plt

# Resto del código ...

# Llamar a la función buscar_producto_mas_consumido para obtener recomendaciones
recomendaciones_resultado = buscar_producto_mas_consumido(kunnr)
print(recomendaciones_resultado)

# Verificar si recomendaciones_resultado es None
if recomendaciones_resultado is not None:
    # Crear el histograma del resultado de recomendaciones
    plt.hist(recomendaciones_resultado, bins=10)  # Ajusta el número de bins según tus necesidades
    plt.xlabel('Valores')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Recomendaciones')

    # Mostrar el histograma
    plt.show()
else:
    print("No se encontraron recomendaciones.")

